In [39]:
%matplotlib inline

import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from scipy.stats import randint, uniform
from sklearn import linear_model, preprocessing
from sklearn.decomposition import PCA
from sklearn.ensemble import (AdaBoostClassifier, BaggingClassifier,
                              RandomForestClassifier, RandomForestRegressor)
from sklearn.linear_model import Lasso, LogisticRegression, Ridge
from sklearn.metrics import (accuracy_score, classification_report,
                             confusion_matrix, mean_absolute_error, r2_score)
from sklearn.model_selection import (GridSearchCV, RandomizedSearchCV,
                                     train_test_split)
from sklearn.preprocessing import (LabelEncoder, MinMaxScaler, OneHotEncoder,
                                   PolynomialFeatures, RobustScaler,
                                   StandardScaler)
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from skopt import BayesSearchCV

pd.set_option('display.max_rows',1000)
pd.set_option('display.max_columns',1000)

In [40]:
# import data
dataset_og = pd.read_csv('Data\Measurements-Transformed')
# kopie maken indien we iets van de originele data nodig hebben
dataset = dataset_og.copy()
dataset.head(5)

,ID,Sex,Measurement_Age,Add,Sph-Far-R,Cyl-Far-R,Axis-Far-R,Sph-Close-R,Cyl-Close-R,Axis-Close-R,Sph-Far-L,Cyl-Far-L,Axis-Far-L,Sph-Close-L,Cyl-Close-L,Axis-Close-L
0,177618.0,0.0,57.0,0.0,-1.75,0.50,55.0,-2.25,1.0,55.0,-1.75,1.00,110.0,-1.25,0.5,110.0
1,177618.0,0.0,55.0,0.0,-1.75,0.50,65.0,0.00,0.0,0.0,-1.25,0.50,110.0,0.00,0.0,0.0
2,177618.0,0.0,49.0,0.0,-1.50,0.50,65.0,0.00,0.0,0.0,-1.00,0.50,110.0,0.00,0.0,0.0
3,278997.0,1.0,37.0,0.0,-3.50,1.50,180.0,-3.50,1.5,180.0,-3.00,1.50,180.0,-3.00,1.5,180.0
4,278997.0,1.0,26.0,0.0,-2.00,0.75,175.0,0.00,0.0,0.0,-2.00,0.75,180.0,0.00,0.0,0.0


### Eerst proberen per feat een model te maken

*bij verandering van target gewoon onderstaande var veranderen dan zou de rest moeten updaten*

In [41]:
# Splitsen in features en targets
target = 'Sph-Far-R'

y = dataset[target].values
X = dataset.drop(target, axis=1)

In [42]:
# Splitsen in train en test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42069)

In [43]:
# Linear regression
lregModel = linear_model.LinearRegression()
lregModel.fit(X_train,y_train)
print('r2 score = ', lregModel.score(X_test,y_test))

r2 score =  0.9254163969654956


In [44]:
# test hogere orde features
graad = 2

polyModel = PolynomialFeatures(graad)
polyModel.fit(X_train)
X_train_poly = polyModel.transform(X_train)
X_test_poly = polyModel.transform(X_test)
print('dimensie van X_train_poly: ', X_train_poly.shape)


# L2 regularisatie via Ridge regression
lregPolyModel = Ridge(alpha=0.5, tol=0.0001, fit_intercept=True)
lregPolyModel.fit(X_train_poly,y_train)

print('R2 score op training set via L2: ', lregPolyModel.score(X_train_poly,y_train))
print('R2 score op test set via L2: ', lregPolyModel.score(X_test_poly,y_test))
print('Intercept score op test set via L2: ', lregPolyModel.intercept_)

dimensie van X_train_poly:  (41893, 136)
R2 score op training set via L2:  0.8962263006243428
R2 score op test set via L2:  0.9347614620029864
Intercept score op test set via L2:  -0.007152223814797243


In [45]:
# Aanmaken van de hogere orde features
RFR_model = RandomForestRegressor(n_estimators=100)
RFR_model.fit(X_train,y_train)

RFR_model.score(X_test,y_test)

0.9269727171658353

### Sliding window

In [46]:
# # Sliding window
# window_size = 2
# X_window = []
# y_window = []

# for w in range(window_size, X.shape[0]):
#     X_window.append(X[w-window_size:w,:].reshape(1,-1))
#     y_window.append(y[w-1])

# X_window = np.asarray(X_window)
# X_window = np.squeeze(X_window, axis=1)
# y_window = np.asarray(y_window)

# X_train, X_test, y_train, y_test = train_test_split(X_window, y_window, test_size=0.2, random_state=0 )

# # Aanmaken van de hogere orde features
# RFR_model.fit(X_train,y_train)
# RFR_model.score(X_test,y_test)

### Rijen aan elkaar plakken

In [48]:
# Drop waardes die minder dan n* records hebben, drops ~10K records, kan mss wat efficienter
# for rec in dataset.ID:
#     if not len(dataset.loc[dataset['ID'] == rec].index) > 1:
#         dataset = dataset[dataset['ID'] != rec]

# kon idd efficienter
prefAmountRecords = 2
# dataset = dataset.groupby('ID').filter(lambda x: len(x) > prefAmountRecords - 1)
# subset = dataset.groupby('ID').filter(lambda x: len(x) < prefAmountRecords + 1)
for group in dataset.groupby('ID'):
    if not len(group) == 2:
        print(len(group))

2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2


In [ ]:
dataset = dataset.merge(dataset, on=['ID', 'Sex'], suffixes=[f'_{i}' for i in range(0, prefAmountRecords + 1)])
dataset = dataset.sort_values(by=['ID', 'Measurement_Age'])
# dataset = dataset.drop_duplicates(subset=['ID', 'Sex'], keep='first')
# /\ verkeerd, deze dropt alle ROWS waar ID en sex duplicates zijn, houd enkel 1 rij van bepaalde ID over
dataset = dataset.loc[:,~df.columns.duplicated()] # https://stackoverflow.com/questions/14984119 

dataset.head(10)

In [ ]:
# Splitsen in features en targets
y = dataset[f'{target}_{prefAmountRecords}'].values
x = dataset[col for col in dataset.columns if not f'_{prefAmountRecords}' in col]

In [ ]:
# Splitsen in training set en test set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=0)

In [ ]:
# Linear regression

regmodel = linear_model.LinearRegression()
regmodel.fit(X_train,y_train)
print(regmodel.coef_)
r2 = regmodel.score(X_test,y_test)
print('r2 score = ', r2)